# Imports

In [1]:
from ultralytics import YOLO
import numpy as np
import eval_helper as help

In [2]:
# !rm -rf /home/andrew/ITRI-LungCancer/outputs
# !mkdir /home/andrew/ITRI-LungCancer/outputs/
# !mkdir /home/andrew/ITRI-LungCancer/outputs/{imgs,histograms}

In [3]:
blob_container_client = help.get_container_client()
model = YOLO('/home/andrew/ITRI-LungCancer/YOLO/model_classify_rgb.pt')
img_path = '/home/andrew/ITRI-LungCancer/outputs/imgs'
hist_path = '/home/andrew/ITRI-LungCancer/outputs/histograms'
final_results = []
starting_patient_id = 544
starting_patient_id = 1
# starting_patient_id = 556
# starting_patient_id = 984

# Go though test patients
iterations = 1000
for patient_index in range(iterations):
    
    # Arrays to track labels
    image_paths = []
    labels_boxes = []
    labels_cls = []
    preds = []
    
    # Go through each patients' blob files (DICOM)
    patient_id = f'LIDC-IDRI-{starting_patient_id+patient_index:04}'
    blob_list = help.get_blob_list(blob_container_client, patient_id)
    for i, blob_name in enumerate(blob_list):
        
        # Retrieve dicom then get slice depth
        dicom = help.get_dicom(blob_container_client, blob_name)
        slice_depth = dicom.ImagePositionPatient[2]
        
        # Get annotations then add cls and bbox
        anns = help.get_anns_from_slice(patient_id, slice_depth)
        help.add_cls_from_anns(labels_cls, i, anns)
        label = help.ann_to_bbox(anns)
        labels_boxes.append(label)
        
        # Stack three slices for input
        image_base = help.dicom_to_img(dicom)
        image_prev, image_next = help.get_context_imgs(blob_container_client, blob_name)
        image = np.stack([image_prev, image_base, image_next], axis=-1)
        
        # Inference then save boxes
        result = model(image, verbose=False, iou=0, conf=0.1)[0]
        preds.append(help.getBBoxes(result))
        
        # Save images to sample_out
        filename = f'{img_path}/{patient_id}_{i}.png'
        help.draw_boxes(filename, result, anns)
    
    # Arrays to filter bboxes
    indices = []
    preds_filtered = []
    labels_filtered = []

    for i, (pred, label) in enumerate(zip(preds, labels_boxes)):
        # Remove slices that don't have enough contiguous slices
        if help.get_num_relevant_slices(i, preds) < 4:
            pred = []
        if help.get_num_relevant_slices(i, labels_boxes) < 4:
            label = []
        
        # Ignore null entries
        if pred == [] and label == []:
            continue
        
        # Add to arrays
        indices.append(i)
        preds_filtered.append(pred)
        labels_filtered.append(label)
        
    # Extract nodules from indices and save to results array
    p_nods = help.get_nodules(indices, preds_filtered)
    l_nods = help.get_nodules(indices, labels_filtered)
    final_results.append((p_nods, l_nods))
    
    # Save histogram of nodules
    help.save_histogram(hist_path, indices, preds_filtered, labels_filtered, patient_id)
    
    prec, rec, f1 = help.evaluate(final_results)
    print(f'Finished {patient_id}, {iterations-patient_index-1} remaining. Precision: {prec:.03f}, Recall: {rec:.03f}, F1 Score: {f1:.03f}', end='\r')